In [3]:
import calendar
from scipy.stats import chi2_contingency
months = list(map(lambda x: x.lower(), calendar.month_name))[1:]
import pandas as pd
import statsmodels.api as sm
from pandas import DataFrame

import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

import matplotlib.pylab as pylab

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 10),
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
pylab.rcParams.update(params)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os

project_folder = f'{os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))}/data/process'

import warnings

warnings.filterwarnings(action='ignore')
from IPython.display import display
from IPython.display import Markdown
from IPython.display import display_html
from python.notebooks.paper_2_notebook.my_classes import MyTests, PreTest
is_ca = 'is_ca'
cal_al = 'cal_al'
channel_count = 'channel_count'
questions ={'Q2':'part','Q6':'day_time','Q7':'is_locked','Q8':'locked_type','Q9':'theft_loc','Q10':'is_registered','Q11':'where_rep','Q12':'is_insured','Q13':'value_approx','Q14':'is_electric','Q15':'bicycle_type','Q17':'is_univ','Q18':'is_recover','Q19':'is_online','Q20':'how_recover','Q21':'is_police','Q23':'recovery_loc','Q24':'rec_cond','Q34_1': 'age_group', 'Q35': 'gender', 'Q36': 'income','Q37':'numb_bikes ', 'Q38': 'education', 'Q39': 'ethnic_origin','state':'state','country':'country','city':'city',channel_count:channel_count,cal_al:cal_al,is_ca:is_ca}
cols_names = list(questions.values())
rows_to_delete = ['I prefer to not answer','Don’t know/not sure']

better_name = {col_val:col_val.replace('_', ' ').capitalize() for col_val in cols_names[1:]}
better_name['day_time'] = 'Time of day'

better_name

{'day_time': 'Time of day',
 'is_locked': 'Is locked',
 'locked_type': 'Locked type',
 'theft_loc': 'Theft loc',
 'is_registered': 'Is registered',
 'where_rep': 'Where rep',
 'is_insured': 'Is insured',
 'value_approx': 'Value approx',
 'is_electric': 'Is electric',
 'bicycle_type': 'Bicycle type',
 'is_univ': 'Is univ',
 'is_recover': 'Is recover',
 'is_online': 'Is online',
 'how_recover': 'How recover',
 'is_police': 'Is police',
 'recovery_loc': 'Recovery loc',
 'rec_cond': 'Rec cond',
 'age_group': 'Age group',
 'gender': 'Gender',
 'income': 'Income',
 'numb_bikes ': 'Numb bikes ',
 'education': 'Education',
 'ethnic_origin': 'Ethnic origin',
 'state': 'State',
 'country': 'Country',
 'city': 'City',
 'channel_count': 'Channel count',
 'cal_al': 'Cal al',
 'is_ca': 'Is ca'}

In [4]:
# Change col names and leave only relevant cols as well as delete unnecessary spaces and parenthesis (except age group)

data_init = pd.read_csv(f'{project_folder}/new_data/new_data_2.csv')
d_analysis = data_init.rename(columns=questions)[cols_names[:-3]].astype(str).replace(r"\(.*?\)", "").astype(str).replace(":", "").apply(lambda row: [d.split('(')[0].strip() if '(' in d else d.strip() for d in row])
# We utilized the original data for the variable 'where_rep'.
d_analysis['where_rep'] = data_init['Q11'].astype(str).replace(r'\(.*?\)', '', regex=True)
# sum the number of channels respondents use
d_analysis['channel_count'] = d_analysis.apply(lambda row: row['where_rep'].count(',') + 1 if row['where_rep'] not in ['I did not report the theft','nan'] else 0, axis=1)

# We have added a new field to check whether the state is California.
d_analysis[is_ca]= d_analysis['state'].apply(lambda a: 'CA'if a== 'California' else 'No_CA')
d_analysis.to_csv(f'{project_folder}/new_data/new_data_3_t1.csv')
d_analysis

,part,day_time,is_locked,locked_type,theft_loc,is_registered,where_rep,is_insured,value_approx,is_electric,...,gender,income,numb_bikes,education,ethnic_origin,state,country,city,channel_count,is_ca
0,Entire bicycle,Overnight,"No, unlocked",nan,Parking garage bike rack/railing/pipe/etc.,Yes,The police,Don’t know/not sure,$1000-$1999,"Yes, with pedal-assist and throttle",...,Woman,"Between $20,000 and $34,999 per year",One,Graduated high school,European,Alberta,CAN,Edmonton,1,No_CA
1,Entire bicycle,Afternoon,"No, unlocked",nan,Other,No,"The police ,A bicycle registry/recovery system...",No,$2000-3999,"Yes, with pedal-assist and throttle",...,Man,"Between $20,000 and $34,999 per year",Three,Some university,European,California,USA,Santa Ana,4,CA
2,Entire bicycle,Overnight,"No, unlocked",nan,Inside a shed/garage,No,"The police ,A bicycle registry/recovery system",Yes,$2000-3999,​No,...,Man,"Between $100,000 and $149,999 per year",Two,Graduate degree,European,California,USA,San Jose,2,CA
3,Entire bicycle,Morning,"Yes, frame locked to a fixed structure",​U-lock,"Outside in the yard or on a balcony, porch, or...",No,The police,Yes,$500-$999,​No,...,Woman,I prefer to not answer,One,Graduate degree,European,Alberta,CAN,Edmonton,1,No_CA
4,Entire bicycle,Morning,"Yes, frame locked to a fixed structure",Cable lock,Other,Yes,"The police ,A bicycle registry/recovery system",Yes,$1000-$1999,​No,...,Man,"Between $100,000 and $149,999 per year",Two,Graduate degree,European,Washington,USA,Seattle,2,No_CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,Entire bicycle,Afternoon,"No, unlocked",nan,Inside a shed/garage,Yes,"The police ,A bicycle registry/recovery system...",Yes,$2000-3999,"Yes, with pedal-assist and throttle",...,nan,nan,nan,nan,nan,California,USA,Napa,4,CA
1817,A major bicycle part necessary for riding,Afternoon,"Yes, frame locked to a fixed structure",Cable lock,Outdoor bike rack,Don’t know/not sure,I did not report the theft,No,$250-$499,nan,...,Prefer not to say,I prefer to not answer,Three,Graduate degree,I prefer to not answer,California,USA,Santa Barbara,0,CA
1818,Entire bicycle,Don’t know/not sure,"Yes, frame locked to a fixed structure",​U-lock,Other,No,The police,No,$500-$999,​No,...,Woman,"Under $20,000 per year",Two,Graduate degree,Multi-ethnic,California,USA,Santa Barbara,1,CA
1819,Entire bicycle,Evening,"No, unlocked",nan,Inside a shed/garage,No,"The police ,A bicycle registry/recovery system...",No,$2000-3999,​No,...,nan,nan,nan,nan,nan,Alberta,CAN,Calgary,3,No_CA


In [17]:
value_re_index = ['Less than $250', '$250-$499', '$500-$999','$1000-$1999','$2000-3999', '$4000-6999','$7000 or more']

In [22]:
# spearmanr
val_1 = 'value_approx'
vals_2 = ['channel_count','day_time','theft_loc','is_insured','is_locked']
d_analysis= pd.read_csv(f'{project_folder}/new_data/new_data_3_t1.csv')
d_analysis_spearmanr_0 = d_analysis[[val_1]+vals_2]
d_analysis_spearmanr = d_analysis_spearmanr_0[~d_analysis_spearmanr_0[val_1].isin(rows_to_delete)]

In [25]:

out = ['Parking garage bike rack/railing/pipe/etc.','Outdoor bike rack','Against a street sign, tree, or street furniture','Building lobby']
my_in= ['Inside a shed/garage','Outside in the yard or on a balcony, porch, or patio','Controlled-access bicycle room or enclosed bike locker','Condo storage room','Inside house/apartment','Workplace office']
d_analysis_spearmanr['where'] =d_analysis_spearmanr['theft_loc'].apply(lambda a: 'out' if a in out else 'in' if a in my_in else a)
d_analysis_spearmanr[['theft_loc','where']]

,theft_loc,where
0,Parking garage bike rack/railing/pipe/etc.,out
1,Other,Other
2,Inside a shed/garage,in
3,"Outside in the yard or on a balcony, porch, or...",in
4,Other,Other
...,...,...
1816,Inside a shed/garage,in
1817,Outdoor bike rack,out
1818,Other,Other
1819,Inside a shed/garage,in


In [26]:

contingency_table = (pd.crosstab(d_analysis_spearmanr['where'], d_analysis_spearmanr[val_1], normalize='columns') * 100).reindex(columns=value_re_index)
contingency_table


value_approx,Less than $250,$250-$499,$500-$999,$1000-$1999,$2000-3999,$4000-6999,$7000 or more
where,,,,,,,
Other,6.493506,11.475410,11.730769,15.081206,16.107383,24.489796,26.315789
in,55.844156,48.360656,50.961538,54.292343,60.738255,61.904762,63.157895
out,37.662338,40.163934,37.307692,30.626450,23.154362,13.605442,10.526316


In [30]:
import numpy as np
import scipy.stats as stats
data_1 = np.array(range(len(contingency_table.columns)))
data_2 = np.array(contingency_table.iloc[2])
result = stats.spearmanr(data_1, data_2)
test_statistic = result.correlation
p_value = result.pvalue
print(data_2)
# Print the test statistic and p-value
print("Test Statistic:", test_statistic)
print("p-value:", p_value)

[37.66233766 40.16393443 37.30769231 30.62645012 23.15436242 13.60544218
 10.52631579]
Test Statistic: -0.9642857142857145
p-value: 0.0004541491691941689


In [33]:
# channel_count and recovery
var_2 = 'channel_count'
pre_test = PreTest(rows_to_delete=[],data_to_explore=pd.read_csv(f'{project_folder}/new_data/new_data_3_t1.csv'),cols_to_explore=cols_names[:-1],more_files_path=[])
MyTests('is_recover','channel_count',('apply_chi',{}),pre_test)


is_recover::channel_count
apply_chi
channel_count          0          1         2          3          4
is_recover                                                         
No             88.709677  88.732394  83.28125  82.972136  81.395349
Yes            11.290323  11.267606  16.71875  17.027864  18.604651
Chi-square test statistic: 3.7488894110161874
p-value: 0.4410552200575599


In [32]:
cols_names

['part',
 'day_time',
 'is_locked',
 'locked_type',
 'theft_loc',
 'is_registered',
 'where_rep',
 'is_insured',
 'value_approx',
 'is_electric',
 'bicycle_type',
 'is_univ',
 'is_recover',
 'is_online',
 'how_recover',
 'is_police',
 'recovery_loc',
 'rec_cond',
 'age_group',
 'gender',
 'income',
 'numb_bikes ',
 'education',
 'ethnic_origin',
 'state',
 'country',
 'city',
 'is_ca',
 'channel_count',
 'cal_al']

In [9]:
# registration and recovery
var_2 = 'is_registered'
pre_test = PreTest(rows_to_delete=[],data_to_explore=pd.read_csv(f'{project_folder}/new_data/new_data_3_t1.csv'),cols_to_explore=cols_names[:-5],more_files_path=[])
MyTests('is_recover','is_registered',('apply_chi',{var_2:{'Don’t know/not sure':"No" }}),pre_test)


is_recover::is_registered
apply_chi
is_registered         No        Yes
is_recover                         
No             86.475063  81.012658
Yes            13.524937  18.987342
Chi-square test statistic: 0.7313707986337676
p-value: 0.3924392198711416


In [12]:
# California VS all the rest
pre_test = PreTest(rows_to_delete=[],data_to_explore=pd.read_csv(f'{project_folder}/new_data/new_data_3_t1.csv'),cols_to_explore=cols_names,more_files_path=[])
res = [MyTests(my_var,is_ca,('apply_chi',{}),pre_test) for my_var in cols_names[:-4]]



part::is_ca
apply_chi
is_ca                                            CA      No_CA
part                                                          
A major bicycle part necessary for riding   1.55587   1.182894
Entire bicycle                             98.44413  98.817106
Chi-square test statistic: 0.0
p-value: 1.0

day_time::is_ca
apply_chi
is_ca                       CA      No_CA
day_time                                 
Afternoon            18.644068  20.271493
Don’t know/not sure  16.242938  19.276018
Evening              14.548023  10.950226
Morning               8.898305   9.773756
Overnight            41.666667  39.728507
Chi-square test statistic: 0.9219077292567426
p-value: 0.9213969742941743

is_locked::is_ca
apply_chi
is_ca                                          CA      No_CA
is_locked                                                   
Don’t know/not sure                      1.128350   2.350814
No, unlocked                            37.940762  40.054250
Yes, frame loc

In [6]:
res[0].constant_data.res

{'part': 1.0,
 'day_time': 0.9213969742941743,
 'is_locked': 0.7896117174647685,
 'locked_type': 0.99999999899743,
 'theft_loc': 0.9991201032294608,
 'is_registered': 0.7295720312347933,
 'where_rep': 0.9978575629403951,
 'is_insured': 0.8762556402077982,
 'value_approx': 0.9389713967502722,
 'is_electric': 0.4171860779873511,
 'bicycle_type': 0.8534614013368115,
 'is_univ': 1.0,
 'is_recover': 0.048979149336396244,
 'is_online': 0.10165709752419329,
 'how_recover': 0.015008322871004523,
 'is_police': 0.11873374913794736,
 'recovery_loc': 0.7807571744884271,
 'rec_cond': 0.001439754823323504,
 'age_group': 0.9994634865637848,
 'gender': 0.7713232411973753,
 'income': 0.5905862980427723,
 'numb_bikes ': 0.9987447172505067,
 'education': 0.9428605857949436,
 'ethnic_origin': 0.971185620006296}

In [35]:
# California VS Alberta - registery
data_to_explore_0 = pd.read_csv(f'{project_folder}/new_data/new_data_3_t1.csv')
data_to_explore = data_to_explore_0[data_to_explore_0['is_registered']=='Yes']
data_to_explore

,Unnamed: 0,part,day_time,is_locked,locked_type,theft_loc,is_registered,where_rep,is_insured,value_approx,...,gender,income,numb_bikes,education,ethnic_origin,state,country,city,channel_count,is_ca
0,0,Entire bicycle,Overnight,"No, unlocked",NaN,Parking garage bike rack/railing/pipe/etc.,Yes,The police,Don’t know/not sure,$1000-$1999,...,Woman,"Between $20,000 and $34,999 per year",One,Graduated high school,European,Alberta,CAN,Edmonton,1,No_CA
4,4,Entire bicycle,Morning,"Yes, frame locked to a fixed structure",Cable lock,Other,Yes,"The police ,A bicycle registry/recovery system",Yes,$1000-$1999,...,Man,"Between $100,000 and $149,999 per year",Two,Graduate degree,European,Washington,USA,Seattle,2,No_CA
5,5,Entire bicycle,Afternoon,"No, unlocked",NaN,Other,Yes,"The police ,A bicycle registry/recovery system",No,$1000-$1999,...,Man,"Between $50,000 and $74,999 per year",Two,Graduate degree,European,Colorado,USA,Boulder,2,No_CA
6,6,Entire bicycle,Afternoon,"Yes, frame locked to a fixed structure",Cable lock,Outdoor bike rack,Yes,The police,No,$2000-3999,...,Woman,"Under $20,000 per year",One,Graduate degree,European,California,USA,San Francisco,1,CA
7,7,Entire bicycle,Overnight,"Yes, locked one or more wheels","​U-lock ,Cable lock",Inside a shed/garage,Yes,"The police ,A bicycle registry/recovery system",Yes,$7000 or more,...,Man,NaN,Two,Graduate degree,NaN,Oregon,USA,Portland,2,No_CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,1800,Entire bicycle,Don’t know/not sure,"Yes, frame locked to a fixed structure",Cable lock,Inside a shed/garage,Yes,"The police ,A bicycle registry/recovery system...",Yes,$1000-$1999,...,NaN,NaN,NaN,NaN,NaN,California,USA,San Francisco,3,CA
1803,1803,Entire bicycle,Afternoon,"Yes, frame locked to a fixed structure",Chain-lock combination,Outdoor bike rack,Yes,The police,No,$500-$999,...,NaN,NaN,NaN,NaN,NaN,California,USA,Emeryville,1,CA
1813,1813,Entire bicycle,Overnight,"No, unlocked",NaN,Inside house/apartment,Yes,"The police ,A bicycle registry/recovery system",Yes,$500-$999,...,NaN,NaN,NaN,NaN,NaN,California,USA,San Francisco,2,CA
1815,1815,Entire bicycle,Overnight,"Yes, frame locked to a fixed structure",Two U-Locks,Controlled-access bicycle room or enclosed bik...,Yes,The police,Yes,$1000-$1999,...,NaN,NaN,NaN,NaN,NaN,Washington,USA,Seattle,1,No_CA


In [37]:
pre_test = PreTest(rows_to_delete=[state for state in data_to_explore['state'].unique() if state not in ['California','Alberta']],data_to_explore=data_to_explore,cols_to_explore=cols_names[:-3],more_files_path=[])
res = MyTests('is_recover','state',('apply_chi',{}),pre_test)


is_recover::state
apply_chi
state         Alberta  California
is_recover                       
No          72.527473   86.382979
Yes         27.472527   13.617021
Chi-square test statistic: 5.062027812318254
p-value: 0.02445560717296731


In [69]:
# California VS Alberta
data_to_explore = pd.read_csv(f'{project_folder}/new_data/new_data_3_t1.csv')
pre_test = PreTest(rows_to_delete=[state for state in data_to_explore['state'].unique() if state not in ['California','Alberta']],data_to_explore=data_to_explore,cols_to_explore=cols_names[:-3],more_files_path=[])
res = [MyTests(my_var,'state',('apply_chi',{}),pre_test) for my_var in cols_names[:-5]]


part::state
apply_chi
state                                      Alberta  California
part                                                          
A major bicycle part necessary for riding      0.0     1.55587
Entire bicycle                               100.0    98.44413
Chi-square test statistic: 0.2001542149938443
p-value: 0.6545963969117644

day_time::state
apply_chi
state                  Alberta  California
day_time                                  
Afternoon            23.504274   18.644068
Don’t know/not sure  11.965812   16.242938
Evening               6.837607   14.548023
Morning              11.965812    8.898305
Overnight            45.726496   41.666667
Chi-square test statistic: 4.628475697972406
p-value: 0.32758419873053324

is_locked::state
apply_chi
state                                     Alberta  California
is_locked                                                    
Don’t know/not sure                      2.136752    1.128350
No, unlocked                       

In [42]:
# Variation by cities
north_ca  =['Davis', 'San Francisco', 'Oakland', 'Palo Alto','San Jose']
south_ca = ['Santa Barbara', 'Santa Monica', 'Los Angeles', 'San Diego']
alberta = ['Edmonton','Calgary']
data_to_explore = pd.read_csv(f'{project_folder}/new_data/new_data_3_t1.csv')
# get only the relevant data
data_to_explore_city = data_to_explore[data_to_explore['city'].isin(north_ca+south_ca+alberta)]
# group the data into four groups
data_to_explore_city[cal_al]= data_to_explore['city'].apply(lambda f: f if f in alberta else 'north_ca' if f in north_ca else 'south_ca')
data_to_explore_city

,Unnamed: 0,part,day_time,is_locked,locked_type,theft_loc,is_registered,where_rep,is_insured,value_approx,...,gender,income,numb_bikes,education,ethnic_origin,state,country,city,is_ca,cal_al
0,0,Entire bicycle,Overnight,"No, unlocked",NaN,Parking garage bike rack/railing/pipe/etc.,Yes,The police,Don’t know/not sure,$1000-$1999,...,Woman,"Between $20,000 and $34,999 per year",One,Graduated high school,European,Alberta,CAN,Edmonton,No_CA,Edmonton
2,2,Entire bicycle,Overnight,"No, unlocked",NaN,Inside a shed/garage,No,"The police ,A bicycle registry/recovery system...",Yes,$2000-3999,...,Man,"Between $100,000 and $149,999 per year",Two,Graduate degree,European,California,USA,San Jose,CA,north_ca
3,3,Entire bicycle,Morning,"Yes, frame locked to a fixed structure",​U-lock,"Outside in the yard or on a balcony, porch, or...",No,The police,Yes,$500-$999,...,Woman,I prefer to not answer,One,Graduate degree,European,Alberta,CAN,Edmonton,No_CA,Edmonton
6,6,Entire bicycle,Afternoon,"Yes, frame locked to a fixed structure",Cable lock,Outdoor bike rack,Yes,The police,No,$2000-3999,...,Woman,"Under $20,000 per year",One,Graduate degree,European,California,USA,San Francisco,CA,north_ca
8,8,Entire bicycle,Morning,"Yes, frame locked to a fixed structure",Cable lock,Outdoor bike rack,Don’t know/not sure,The police,No,$1000-$1999,...,Man,I prefer to not answer,Two,Bachelor’s degree,European,California,USA,Santa Barbara,CA,south_ca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814,1814,Entire bicycle,Overnight,"No, unlocked",NaN,"Outside in the yard or on a balcony, porch, or...",No,"The police ,A bicycle registry/recovery system...",Yes,$2000-3999,...,NaN,NaN,NaN,NaN,NaN,California,USA,Los Angeles,CA,south_ca
1817,1817,A major bicycle part necessary for riding,Afternoon,"Yes, frame locked to a fixed structure",Cable lock,Outdoor bike rack,Don’t know/not sure,I did not report the theft,No,$250-$499,...,Prefer not to say,I prefer to not answer,Three,Graduate degree,I prefer to not answer,California,USA,Santa Barbara,CA,south_ca
1818,1818,Entire bicycle,Don’t know/not sure,"Yes, frame locked to a fixed structure",​U-lock,Other,No,The police,No,$500-$999,...,Woman,"Under $20,000 per year",Two,Graduate degree,Multi-ethnic,California,USA,Santa Barbara,CA,south_ca
1819,1819,Entire bicycle,Evening,"No, unlocked",NaN,Inside a shed/garage,No,"The police ,A bicycle registry/recovery system...",No,$2000-3999,...,NaN,NaN,NaN,NaN,NaN,Alberta,CAN,Calgary,No_CA,Calgary


In [54]:
pre_test = PreTest(rows_to_delete=[],data_to_explore=data_to_explore_city,cols_to_explore=cols_names,more_files_path=[])
res = [MyTests(my_var,cal_al,('apply_chi',{}),pre_test) for my_var in cols_names[:-5]]


part::cal_al
apply_chi
cal_al                                     Calgary  Edmonton  north_ca  \
part                                                                     
A major bicycle part necessary for riding      0.0       0.0       0.0   
Entire bicycle                               100.0     100.0     100.0   

cal_al                                      south_ca  
part                                                  
A major bicycle part necessary for riding   3.703704  
Entire bicycle                             96.296296  
Chi-square test statistic: 11.214953271028037
p-value: 0.010618554409406

day_time::cal_al
apply_chi
cal_al                 Calgary  Edmonton   north_ca   south_ca
day_time                                                      
Afternoon            22.772277      25.0  13.178295  18.848168
Don’t know/not sure  13.861386       8.0  14.341085  20.418848
Evening               6.930693       7.0  13.565891  14.136126
Morning              13.861386      13.0   

In [95]:
with pd.ExcelWriter(f'{os.path.dirname(os.getcwd())}/output/general/cal_alb.xlsx') as writer:
    for re in cols_names[:-5]:
        if re in  ['is_online','how_recover','is_registered','is_police']:
            tbl = res[0].constant_data.contingency_table[re].sort_values('California',ascending=False).reindex(columns=['California','Alberta'])
            tbl.index.name = better_name[tbl.index.name]
            tbl.to_excel(writer, sheet_name=re)
    # sum the number of channels respondents use
    tbl =res[0].constant_data.contingency_table['where_rep'].reset_index()
    tbl['where_rep']= tbl['where_rep'].astype(str).replace(r'\(.*?\)', '', regex=True)
    tbl['comma_count'] =tbl['where_rep'].str.count(',')
    tbl = tbl.groupby('comma_count').sum().sort_values('California',ascending=False).reindex(columns=['California','Alberta'])
    tbl.index.name = '# channels'
    tbl.to_excel(writer, sheet_name='channels')




In [96]:
tbl =res[0].constant_data.contingency_table['where_rep'].reset_index()
tbl['where_rep']= tbl['where_rep'].astype(str).replace(r'\(.*?\)', '', regex=True)
tbl

state,where_rep,Alberta,California
0,A bicycle registry/recovery system,6.837607,6.488011
1,"A bicycle registry/recovery system ,Other :",1.282051,0.141044
2,"A bicycle registry/recovery system ,Social media",2.564103,1.551481
3,"A bicycle registry/recovery system ,Social med...",0.000000,0.141044
4,I did not report the theft,1.282051,3.949224
5,Other :,0.000000,0.987306
6,Social media,0.000000,0.282087
7,The police,14.102564,17.630465
8,"The police ,A bicycle registry/recovery system",32.051282,32.016925
9,"The police ,A bicycle registry/recovery system...",0.427350,1.551481


In [ ]:
res[0].constant_data.contingency_table['where_rep']

In [94]:
# sum the number of channels respondents use
tbl =res[0].constant_data.contingency_table['where_rep'].reset_index()
tbl['where_rep']= tbl['where_rep'].astype(str).replace(r'\(.*?\)', '', regex=True)
tbl['comma_count'] =tbl['where_rep'].str.count(',')
tbl = tbl.groupby('comma_count').sum().sort_values('California',ascending=False).reindex(columns=['California','Alberta'])
tbl.index.name = '# channels'
with pd.ExcelWriter(f'{os.path.dirname(os.getcwd())}/output/general/cal_alb.xlsx') as writer:
    tbl.to_excel(writer, sheet_name='channels')